# Caesar shift.

## Задача №1.

1. Написать алгоритм шифра Цезаря для генерации выборки (сдвиг на К каждой буквы. Например, при сдвиге на 2 буква “А” переходит в букву “В” и тп)
2. Сделать нейронную сеть
3. Обучить ее (вход - зашифрованная фраза, выход - дешифрованная фраза)
4. Проверить качество

In [105]:
import re
import torch
from tqdm import tqdm

In [7]:
# import torchtext.utils

In [8]:
letters = "а б в г д е ё ж з и й к л м н о п р с т у ф х ц ч ш щ ъ ы ь э ю я"

In [9]:
indexes = [i for i in range(33)]

In [10]:
alphabet = dict(zip(indexes, letters.split()))

In [11]:
# pattern = r'[АБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯабвгдеёжзийклмнопрстуфхцчшщъыьэюя]'
pattern = r'[абвгдеёжзийклмнопрстуфхцчшщъыьэюя]'

In [12]:
# list(alphabet.values()).index('я')

In [13]:
# def get_letter(index):
#     return alphabet[index]

In [14]:
def get_index(letter):
    if letter in alphabet.values():
        return list(alphabet.values()).index(letter)

In [15]:
type(get_index('б'))

int

In [16]:
# Если сдвиг 5
shift = 7
alphabet[get_index('п')]

'п'

In [17]:
def get_new_index(alphabet_lenght, index, shift):
    if shift >= alphabet_lenght:
        shift = shift%alphabet_lenght
    else:
        pass
    
    if index < shift:
        return alphabet_lenght + (index - shift%alphabet_lenght)
    else:
        return index - shift%alphabet_lenght

In [18]:
test_string = "Некоторые люди для запоминания количества букв в русском алфавите ассоциируют их с популярными фразами: «33 удовольствия», «33 несчастья», «33 коровы». Другие люди ассоциируют с фактами из своей жизни: живу в квартире номер 33, живу в регионе 33 (Владимирская область), играю в команде под номером 33 и подобными. И если число букв алфавита вновь забывается, то ассоциированные фразы помогают его вспомнить. Может, и вам поможет."

In [19]:
test_string.lower()

'некоторые люди для запоминания количества букв в русском алфавите ассоциируют их с популярными фразами: «33 удовольствия», «33 несчастья», «33 коровы». другие люди ассоциируют с фактами из своей жизни: живу в квартире номер 33, живу в регионе 33 (владимирская область), играю в команде под номером 33 и подобными. и если число букв алфавита вновь забывается, то ассоциированные фразы помогают его вспомнить. может, и вам поможет.'

In [20]:
new_string = []

for letter in test_string.lower():
    if letter in re.findall(pattern=pattern, string=test_string.lower()):
        index = get_new_index(len(alphabet), get_index(letter), shift)
        letter = alphabet[index]
    else:
        pass
    new_string.append(letter)

In [49]:
new_string = ''.join(new_string)

In [22]:
file = open('./onegin.txt', 'r')

In [23]:
lines = file.readlines()

In [24]:
original = lines[21:].copy()
encrypted = []

In [25]:
#Удаляем табуляции, знаки переноса и пробелы по краям строк. 
for item in original:
    item.strip()

In [26]:
max( [len(i) for i in original] )

260

In [27]:
# Кодируем строки

for item in original:
    tmp = []
    item.lower().strip()
    for letter in item:
        if letter in re.findall(pattern=pattern, string=item):
            index = get_new_index(len(alphabet), get_index(letter), shift)
            letter = alphabet[index]
        else:
            pass
        tmp.append(letter)
    
    encrypted.append(''.join(tmp))

In [28]:
max( [len(i) for i in encrypted] )

260

In [29]:
len(original), len(encrypted)

(1996, 1996)

In [30]:
# Разобъем данные на обучающую и тестовую выборки.
from sklearn.model_selection import train_test_split

In [31]:
from sklearn.feature_extraction.text import CountVectorizer

In [32]:
X_train, X_test, y_train, y_test = train_test_split(encrypted, original, test_size=0.3, random_state=73)

In [63]:
vec = CountVectorizer(analyzer='word', ngram_range=(1,1))

In [67]:
caesar_train = vec.fit_transform(X_train)
pushkin_train = vec.fit_transform(y_train)

In [72]:
caesar_test = vec.fit_transform(X_test)
pushkin_test = vec.fit_transform(y_test)

In [157]:
caesar_train.get_shape(), pushkin_train.get_shape()

((1397, 2596), (1397, 2464))

In [160]:
t = caesar_train.toarray()

In [161]:
t.shape

(1397, 2596)

In [70]:
from torch.utils.data import TensorDataset, DataLoader

In [127]:
# Два набора данных: для обучения и тестирования.
batch_size = 32
dataset_train = TensorDataset(torch.from_numpy(caesar_train.toarray()).float(), torch.from_numpy(pushkin_train.toarray()).float())
dataset_test = TensorDataset(torch.from_numpy(caesar_test.toarray()).float(), torch.from_numpy(pushkin_test.toarray()).float())

In [162]:
# Randomly reading mini-batches
data_train = DataLoader(dataset_train, batch_size, shuffle=True)
data_test = DataLoader(dataset_test, batch_size)

In [167]:
data_train.dataset.tensors[1].shape

torch.Size([1397, 2464])

In [179]:
class CaesarShift(torch.nn.Module):

    def __init__(self):
        super(CaesarShift, self).__init__()
        
        self.encoder = torch.nn.RNN(input_size=2596, hidden_size=128, num_layers=10, dropout = 0.5, batch_first=True)
        
        self.decoder = torch.nn.RNN(input_size=128, hidden_size=2464, num_layers=10, dropout = 0.5, batch_first=True)
        
        
    def forward(self, x):
        encoder, h_0 = self.encoder(x)
        decoder = self.decoder(encoder)
        return decoder

# model
model = CaesarShift()


criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.RMSprop(model.parameters(), lr = 0.0001)

In [180]:
model

CaesarShift(
  (encoder): RNN(2596, 128, num_layers=10, batch_first=True, dropout=0.5)
  (decoder): RNN(128, 2464, num_layers=10, batch_first=True, dropout=0.5)
)

In [181]:
# num_epochs = 20
# for epoch in tqdm(range(num_epochs)):
#     for X, y in data_train:
#         # X, y = X.to(device), y.to(device)
#         pred_y = model(X)
#         loss = loss(pred_y, y)
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()
# #     # считаем loss на тренировочных данных
# # #     loss = criterion(our_model(X_train), y_train)
# # #     loss_per_epoch[epoch] = loss

In [182]:
for epoch in range(num_epochs):
    for batch_idx, (data, target) in enumerate(data_train):
        #forward
        predict, _ = model(data)
        loss = criterion(predict, target)
        
        #backward
        optimizer.zero_grad()
        loss.backward()
        
        #gradient descent or RMSProp step
        optimizer.step()

KeyboardInterrupt: 

## Задача №2.

1. Построить RNN-ячейку на основе полносвязных слоев
2. Применить построенную ячейку для генерации текста с выражениями героев сериала “Симпсоны”